## Finding pole positions

In [ ]:
import numpy as np
from iminuit import Minuit
import matplotlib.pyplot as plt

In [109]:
resonances = [
    {"m0": 0.8, "g0": 0.4, "s_guess": complex(-1, 1)},
    {"m0": 1.0, "g0": 0.7, "s_guess": complex(-1, 1)},
]

In [110]:
def rho_cm(s):
    m1= 0.5
    m2 = 0.1
    s = complex(s)
    return np.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


## Define $T$ matrix

Define $T$ matrix via $K$-matrix:
$$T(s) = \left(\mathbb{I} + i K(s) {\rho_{cm}(s)}\right)^{-1} K(s)$$
For one channel and multiple resonances this gives:\
$$\sum_\mathrm{Res.}\frac{{K(s)}}{i {K(s)} {{\rho_{cm}(s)}} + 1}$$

A resonance pole in the complex $s$ plane occurs when the denominator of $T(s)$ goes to zero:
$$|T(s)| \rightarrow \infty \quad \Rightarrow \quad \left| \frac{1}{1 - i K(s)} \right|^2 \rightarrow \infty$$

Finding the poles is equivalent to minimizing:

$$\left|1 - i K(s)\right|^2 \rightarrow 0$$

to solve the problem numerically one can separate real and complex part of $s$ to construct a cost function which can be given to common minimizer: Minuit2
Cost function:
$$\text{Cost}(x, y) = \left|1 - i K(x + i y)\right|^2$$
using where $x$ and $y$ are minimization parameters.

In [111]:
def D(s, m0, g0):
    return s - m0**2 + 1j * g0**2 * rho_cm(s)

def pole_cost_function(s_re, s_im, m0, g0):
    s = complex(s_re, s_im)
    d = D(s, m0, g0)
    return d.real**2 + d.imag**2


def fit_pole(m0, g0, s_guess):
    def cost(s_re, s_im): return pole_cost_function(s_re, s_im, m0, g0)
    minuit2 = Minuit(cost, s_re=s_guess.real, s_im=s_guess.imag)
    minuit2.errordef = Minuit.LEAST_SQUARES
    return minuit2.migrad()

In [112]:
m0_res1, g0_res1= resonances[0]["m0"], resonances[0]["g0"]
s_guess_res1 = resonances[0]["s_guess"]
fit_res1 = fit_pole(m0_res1, g0_res1, s_guess_res1)
fit_res1

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.32e-06                   │              Nfcn = 30               │
│ EDM = 2.32e-06 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ s_re │    0.6    │    1.0    │            │            │         │         │       │
│ 1 │ s_im │   -0.1    │    1.0    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────┬───────────┐
│      │ s_re s_im │
├──────┼───────────┤
│ s_re │ 1.07  0.0 │
│ s_im │  0.0 1.06 │
└──────┴───────────┘

In [113]:
m0_res2, g0_res2 = resonances[1]["m0"], resonances[1]["g0"]
s_guess_res2 = resonances[1]["s_guess"]
fit_res2 = fit_pole(m0_res2, g0_res2, s_guess_res2)
fit_res2


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 4.367e-05                  │              Nfcn = 30               │
│ EDM = 4.36e-05 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ s_re │    1.0    │    1.1    │            │            │         │         │       │
│ 1 │ s_im │   -0.4    │    1.1    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────┬───────────┐
│      │ s_re s_im │
├──────┼───────────┤
│ s_re │ 1.19 -0.0 │
│ s_im │ -0.0 1.19 │
└──────┴───────────┘